#14_CNN_VGG11&CIFAR10

###ML 라이브러리: Pytorch(GPU)

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
 
from torchvision import models

GPU 사용을 명시적으로 보여주기 & Random Seed 고정하기

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

학습 파라미터 설정

In [3]:
learning_rate = 0.001
training_epochs = 10
batch_size = 32

##1. 데이터셋 불러오기

In [4]:
transform = transforms.Compose(
      [transforms.Resize((224, 224)),
       transforms.ToTensor(), 
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
train_set = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, shuffle = True)

Files already downloaded and verified


In [6]:
test_set = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size, shuffle = False)

Files already downloaded and verified


##5. 모델 추정

###모델 정의: VGG11(사전학습 모델)

In [7]:
vgg11 = models.vgg11(pretrained = True) # pretrained = True : 사전학습 모델 사용한다 default = False
vgg11.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [8]:
print(vgg11)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

CIFAR10의 클래스 개수가 10개 이므로 vgg10가 분류하는 클래스 개수를 10개로 바꿔준다<br>
- vgg11.classifier[6]은 (classifier)의 (6): Linear를 의미한다
- (6): Linear의 out_featur를 1,000에서 10으로 바꿔준다

In [9]:
vgg11.classifier[6].out_features = 10

사전학습 모델 사용시 주의 사항
- 반드시 로드한 가중치는 변경되지 않도록 한다
- VGG11 모델 중 FC 이전까지의 파라미터를 고정한다

In [10]:
# .required_grad = False : 가중치 갱신x(파라미터 고정)
# = True : 가중치 갱신(학습 중)
for parameter in vgg11.features.parameters():
    parameter.required_grad = False

###모델 학습

Cross Entropy Loss & SGD

In [11]:
loss = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(vgg11.classifier.parameters(), lr = learning_rate, momentum = 0.9)

Training Functinon

In [12]:
def train(model, train_data_lodaer):
    model.train()

    train_running_loss = 0.0
    train_running_correct = 0

    for i, data in enumerate(train_data_lodaer):
        data, target = data[0].to(device), data[1].to(device)

        # Gradient Initialization
        optimizer.zero_grad()

        # Forward 계산
        hypothesis = model(data)

        # Cost : Error 계산
        cost = loss(hypothesis, target)

        # Backpropagation
        cost.backward()

        # Cost로 가중치(W, b) 갱신
        optimizer.step()

        train_running_loss += cost.item()
        _, preds = torch.max(hypothesis.data, 1)
        train_running_correct += (preds == target).sum().item()

    train_loss = train_running_loss/len(train_data_lodaer.dataset)
    train_accuracy = 100. * train_running_correct/len(train_data_lodaer.dataset)
      
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

Validation Function

In [15]:
def validate(model, test_data_loader):
    model.eval()

    val_running_loss = 0.0
    val_running_correct = 0

    for int, data in enumerate(test_data_loader):
        data, target = data[0].to(device), data[1].to(device)

        # Forward 계산
        hypothesis = model(data)

        # Cost : Error 계산
        cost = loss(hypothesis, target)

        val_running_loss += cost.item()
        _, preds = torch.max(hypothesis.data, 1)
        val_running_correct += (preds == target).sum().item()
    
    val_loss = val_running_loss / len(test_data_loader.dataset)
    val_accuracy = 100. * val_running_correct / len(test_data_loader.dataset)

    return val_loss, val_accuracy

모델 학습

In [ ]:
train_loss, train_accuracy = [], []
val_loss, val_accuracy = [], []

start = time.time()
for epoch in range(training_epochs):
    # Train
    train_epoch_loss, train_epoch_accuracy = train(vgg11, train_loader)

    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)

    # Validate
    val_epoch_loss, val_epoch_accuracy = validate(vgg11, test_loader)

    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

    print(train_accuracy, val_accuracy)

end = time.time()
print((end - start)/60, 'minute')

##6. 결과 분석

Accuracy

In [ ]:
plt.figure(figsize = (10, 7))
plt.plot(train_accuracy, color = 'green', label = 'train accuracy')
plt.plot(val_accuracy, color = 'blue', label = 'validation accuracy')
plt.legend()
plt.savefid('accuracy.png')
plt.show()

Loss

In [ ]:
plt.figure(figsize = (10, 7))
plt.plot(train_loss, color = 'orange', label = 'train loss')
plt.plot(val_loss, color = 'red', label = 'validation loss')
plt.legend()
plt.savefid('loss.png')
plt.show()